In [8]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import os
from astropy import wcs
from astropy.table import Table, hstack, Column



In [46]:
catalog = Table.read('/Users/hph/Dropbox/astrophys/summer_students/CH2O_CMZoom/megacatalog_team_acc_final_say.fits')
cont_mosaic_hdu = fits.open('/Users/hph/Dropbox/astrophys/CMZoom/cmzoom_catalog/catalog_acc/CMZoom_continuum_pbcor.fits')
cat_mask = fits.getdata('/Users/hph/Dropbox/astrophys/CMZoom/cmzoom_catalog/catalog_acc/dendrogram_mask_pruned_rms3e6_k14_dv3_dd1_dp17_pp6_pm2_gal_02-02-20.fits')
cube_hdu = fits.open('./5205864.fits')
#cube_hdu[0].header
del cube_hdu[0].header['TIMESYS']

In [106]:
cube_hdu[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    3 / number of array dimensions                     
NAXIS1  =                  294                                                  
NAXIS2  =                  311                                                  
NAXIS3  =                  369                                                  
BMAJ    = 0.000942246367684507                                                  
BMIN    = 0.000936702335293851                                                  
BPA     =   -4.416870154674872                                                  
BTYPE   = 'Intensity'                                                           
OBJECT  = 'G0.340+0'                                                            
BUNIT   = 'Jy/beam '           /Brightness (pixel) unit                         
TELESCOP= 'SMA     '        

In [47]:
### construct necessary coordinate systems

cont_wcs = wcs.WCS(cont_mosaic_hdu[0])
cube_wcs = wcs.WCS(cube_hdu[0])
print(cube_wcs)

WCS Keywords

Number of WCS axes: 3
CTYPE : 'GLON-SIN'  'GLAT-SIN'  'VRAD'  
CRVAL : 0.3385594558594  0.0573852309422  205094.860379762  
CRPIX : 175.0  146.0  1.0  
PC1_1 PC1_2 PC1_3  : 1.0  0.0  0.0  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -0.0001388888888889  0.0001388888888889  -1116.980806414887  
NAXIS : 294  311  369


In [61]:
#### for each leaf, identify peak continuum flux pixel
lpp_x = np.zeros_like(catalog['leaf_ID'])
lpp_y = np.zeros_like(catalog['leaf_ID'])
lpp_gal = np.empty(len(catalog['leaf_ID']),dtype=type(cont_wcs.pixel_to_world(0,0)))
lpp_l = np.zeros_like(catalog['leaf_ID'])
lpp_b = np.zeros_like(catalog['leaf_ID'])

for i, leafi in enumerate(catalog['index']):
    leafpixi = np.where(cat_mask==leafi)
    lpp_x[i] = leafpixi[1][np.argmax(cont_mosaic_hdu[0].data[leafpixi])]
    lpp_y[i] = leafpixi[0][np.argmax(cont_mosaic_hdu[0].data[leafpixi])]
    print(lpp_x[i], lpp_y[i])
    lpp_gal[i] = cont_wcs.pixel_to_world_values(int(lpp_x[i]),int(lpp_y[i]))
    print(lpp_gal[i])
    


15214 201
(array(359.62424595), array(-0.24738112))
15282 230
(array(359.61480154), array(-0.24335315))
15260 219
(array(359.61785708), array(-0.24488099))
10180 483
(array(0.32341628), array(-0.20822211))
10185 516
(array(0.32272184), array(-0.20363878))
10242 517
(array(0.31480514), array(-0.20349987))
10236 534
(array(0.31563848), array(-0.20113876))
10263 546
(array(0.31188847), array(-0.19947208))
10232 552
(array(0.31619405), array(-0.19863876))
10253 552
(array(0.31327737), array(-0.19863875))
10243 557
(array(0.31466627), array(-0.19794431))
10230 575
(array(0.31647183), array(-0.19544432))
10254 579
(array(0.31313849), array(-0.19488875))
10229 600
(array(0.31661073), array(-0.1919721))
10249 603
(array(0.31383295), array(-0.19155542))
15229 982
(array(359.62216473), array(-0.13890885))
15098 1015
(array(359.64035931), array(-0.13432587))
15132 1022
(array(359.6356371), array(-0.13335356))
15139 1035
(array(359.63466491), array(-0.13154799))
15146 1035
(array(359.63369268), ar

7864 1679
(array(0.64508334), array(-0.04211047))
7773 1679
(array(0.65772223), array(-0.0421104))
12863 1677
(array(359.95077776), array(-0.0423862))
12950 1681
(array(359.93869443), array(-0.0418305))
7851 1682
(array(0.6468889), array(-0.0416938))
7668 1690
(array(0.67230556), array(-0.04058254))
7784 1688
(array(0.65619445), array(-0.04086041))
12947 1695
(array(359.93911112), array(-0.03988606))
8180 1712
(array(0.60119443), array(-0.03752736))
12863 1696
(array(359.95077779), array(-0.03974731))
12959 1696
(array(359.93744445), array(-0.03974715))
7643 1703
(array(0.67577777), array(-0.03877696))
12841 1699
(array(359.95383335), array(-0.03933068))
12948 1707
(array(359.93897225), array(-0.03821939))
7636 1718
(array(0.67674998), array(-0.03669362))
7665 1723
(array(0.67272219), array(-0.0359992))
12866 1719
(array(359.95036116), array(-0.03655286))
7743 1717
(array(0.66188887), array(-0.0368326))
7707 1729
(array(0.66688886), array(-0.0351659))
7776 1725
(array(0.65730553), arra

In [31]:
### testing for a single leaf
# k, leafk = 0, catalog['index'][0]
# leafpixk = np.where(cat_mask==leafk)
# print(leafpixk)
# print(np.argmax(cont_mosaic_hdu[0].data[leafpixk]))
# kpp_x = leafpixk[1][np.argmax(cont_mosaic_hdu[0].data[leafpixk])]
# kpp_y = leafpixk[0][np.argmax(cont_mosaic_hdu[0].data[leafpixk])]
# print(kpp_x, kpp_y)


In [98]:
### convert peak flux coords for each leaf to sample cube coords
cube_pix = np.zeros_like(lpp_gal)
cube_only_pix_x = np.array([])
cube_only_pix_y = np.array([])
for i, pixi in enumerate(lpp_gal):
    cube_pix[i] = cube_wcs.world_to_pixel_values(pixi[0],pixi[1],0)[:-1]
    #print(i, cube_pix[i][0], cube_pix[i][1])
    ### find subset of leaf peak pixels within the subcube
    if ((cube_pix[i][0]>0) and 
        (cube_pix[i][1]>0) and 
        (cube_pix[i][0]<cube_hdu[0].header['NAXIS1']) and 
        (cube_pix[i][1]>cube_hdu[0].header['NAXIS2'])):
        cube_only_pix_x = np.append(arr=cube_only_pix, values=cube_pix[i][0])
        cube_only_pix_y = np.append(arr=cube_only_pix, values=cube_pix[i][1])
        print(i, catalog['leaf_ID'][i], cube_only_pix_x[-1],cube_only_pix_y[-1])


262 G359.137+0.031c 8775.38447031603 18.860363319612347
272 G0.340+0.055b 213.02670330339578 91.62697882715639
273 G0.340+0.055e 144.02684603031744 112.62684163332852
274 G0.340+0.055f 136.02686615580853 113.62682754066499
276 G0.340+0.055c 147.02681796088874 123.62684698925375
277 G0.340+0.055a 135.02684457077257 127.6268257797567
279 G0.340+0.055d 127.02683845394358 144.62681210535698


In [87]:
print((cube_pix[273][0]>0) and (cube_pix[273][1]>0))
print((cube_pix[273][0]>0) and (cube_pix[273][1]>0))

True
True


In [58]:
cube_pix = cube_wcs.pixel_to_world_values(0,0,0)

In [59]:
print(cube_pix)

(array(0.36272613), array(0.03724634), array(205094.86037976))
